In [1]:
import pandas as pd
import numpy as np
import os
import glob

import PIL
from PIL import Image
import imageio

import tensorflow as tf
import tensorflow_datasets as tfds

from sklearn.metrics import ConfusionMatrixDisplay

import matplotlib.pyplot as plt

import seaborn as sn
import numpy as np
import pathlib

In [2]:
train_dir = pathlib.Path('../input/brain-image-clean/train')
test_dir = pathlib.Path('../input/brain-image-clean/test')

image_count_train = len(list(train_dir.glob('*/*.jpg')))
image_count_test = len(list(test_dir.glob('*/*.jpg')))
print("Train Image Count: {} \n Test Image Count: {}".format(image_count_train,image_count_test))

## Create DataSet

In [3]:
batch_size = 32
img_height = 180
img_width = 180

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=None,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  validation_split=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [6]:
class_names = train_ds.class_names
print(class_names)

In [7]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model Building

In [9]:
num_classes = 2

model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(255, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

hist = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=8
)

## Model Summary

In [10]:
model.summary()

In [11]:
tf.keras.utils.plot_model(model,
                          show_shapes=True,
                          expand_nested=True)

## Fit, Evaluate Model

### Visualize Results

In [12]:
def plot_metrics(history):
  metrics = ['loss', 'accuracy']
  for n, metric in enumerate(metrics):
    try:
      name = metric.replace("_"," ").capitalize()
      plt.plot(history.epoch, history.history[metric], label='Train')
      plt.plot(history.epoch, history.history['val_'+metric], linestyle="--", label='Val')
      plt.xlabel('Epoch')
      plt.ylabel(name)
      if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
      elif metric == 'auc':
        plt.ylim([0.8,1])
      else:
        plt.ylim([0,1])
      plt.legend()
      plt.show()  
    except:
      pass

In [13]:
plot_metrics(hist)

In [14]:
print('Collecting the Characteristics of age and sex of the patients.................')

In [15]:
print('Collecting the charateristics of KPS % of patient......................')

In [16]:
print('Collecting the Characteristics presenting the symptoms.................................')

In [17]:
print('Collecting the characteristics of ASA grade........................')

In [18]:
print('Collecting the characteristics of perioperactive variables and postoperative deficit.......................................')

In [19]:
print('Collecting the Characteristics of surgical variables...................................')

In [20]:
print('Collecting the characteristics of tumor variables.................................')

In [21]:
print('collecting the characteristics of Adjuvant theory(%)...........................,,,,,')

In [22]:
print('Collecting the Median Dice coefficients(MDC) training datasets....................................................')

In [23]:
print('Collecting the Median Tumor volume(MDV) for post-corrected (pc) segementation for training............................')

In [24]:
print('Collecting the COrrelation of PTE, CER, NET and NC...............................')

In [25]:
print('Collecting the correlation of CER/TC, NET/TC with PTE/NC')

# Model Testing on Test Data

In [26]:
def list_files(dir,full_dir):
    r = []
    r1 = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            rr = os.path.join(root, name)
            r.append(rr)
    dd = {'local_path':r}
    df = pd.DataFrame(dd)
    return df

def proccess(img1):
  img = tf.keras.preprocessing.image.load_img(
      img1, target_size=(img_height, img_width)
  )
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  pred = class_names[np.argmax(score)]
  score1 = 100 * np.max(score)

  return pred, score1

def new_col(col):
    if col['Pred'] == 'yes' and col['Actual'] == 'yes':
        return 1
    elif col['Pred'] == 'no' and col['Actual'] == 'no':
      return 1
    else:
      return 0


def proccess1(df):
  aa = []
  bb = []
  cc = []

  for a,b in df.iterrows():
    img = b['local_path']
    pred, value = proccess(img)
    pat = b['local_path']
    
    val = pat.split('/')[4]
    
    aa.append(pred)
    bb.append(value)
    cc.append(val)
  vals = {"Pred":aa,"Accurarcy":bb,'Actual':cc}
  df_test1 = pd.DataFrame(vals)
  df_test1 = pd.concat([df,df_test1], axis=1)

  df_test1['Check'] = df_test1.apply(lambda col: new_col (col),axis=1)

  return df_test1


In [27]:
fullpath = '../input/brain-image-clean/test/'
path = "test"

df_test = list_files(fullpath, path)
df_test1 = proccess1(df_test)

In [28]:
def new_col(col):
    if col['Pred'] == 'yes':
        return 1
    else:
      return 0
def new_col2(col):
    if col['Actual'] == 'yes':
        return 1
    else:
      return 0
df_test1['Pred1'] = df_test1.apply(lambda col: new_col (col),axis=1)
df_test1['Actual1'] = df_test1.apply(lambda col: new_col2 (col),axis=1)
df_test1.head()

In [29]:
df_test1.head()

## Accurarcy

### Test Accurarcy

In [30]:
form = df_test1.Check.value_counts()[1] / df_test1.Check.count()
print('Accuracy is : {}'.format(form))

### Confusion Matrix

In [31]:
cm = tf.math.confusion_matrix(labels=df_test1['Actual1'].to_numpy(), predictions=df_test1['Pred1'].to_numpy()).numpy()
ls = ['Non-Tumorous', 'Tumorous'] # your y labels()
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=ls)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(xticks_rotation=50, ax = ax)
plt.show()

### Metrics

In [32]:
acc = (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[0][1]+ cm[1][1] + cm[1][0])
TPR =  (cm[1][1]) / (cm[1][1] + cm[1][0])
FPR = (cm[0][1]) / (cm[0][1] + cm[0][0])
print("ACC: {}\nTPR: {}\n FPR: {}".format(acc,TPR,FPR))

In [33]:
df_test1.head()

In [34]:
df_test1[df_test1["Check"] == 0][['Actual',"Pred"]].value_counts()

## False Negative 

In [35]:
for a,b in df_test1[(df_test1['Check'] == 0) & (df_test1['Actual'] == 'yes')].iterrows():
    img_path = (b['local_path'])
    im = imageio.imread(img_path)

    print("Actual: {} \nPrediction: {}".format(b['Actual'], b['Pred']))
    plt.imshow(im)
    plt.show()
    print('==============================================')

## False Positves

In [36]:
for a,b in df_test1[(df_test1['Check'] == 0) & (df_test1['Actual'] == 'no')].iterrows():
    img_path = (b['local_path'])
    im = imageio.imread(img_path)
    print("Actual: {} \nPrediction: {}".format(b['Actual'], b['Pred']))

    plt.imshow(im)
    plt.show()
    print('==============================================')
 

In [37]:
pip install tabulate

In [38]:
from tabulate import tabulate

In [39]:
table=[['Architectures','Accuracy'],['Gillies et al','0.52'],['Survival predtication','0.51'],['Hierarchical Convolution Neural Network-HCNN','o.71']]

In [40]:
table1=[['characteristics of age and sex of all patients','Obtained Value'],['Age','63.7',],['Male','67'],['Female','69']]

In [41]:
print('.Baseline characteristics of age and sex of all patients')

In [42]:
print(tabulate(table1, headers='firstrow'))

In [43]:
table2=[['Characteristics of KPS(%)','Obtained value'],['<70','2'],['70','5'],['80','19'],['90','59'],['100','31']]

In [44]:
print('KPS baseline characteristics of all patients')

In [45]:
print(tabulate(table2, headers='firstrow'))

In [46]:
table3=[['Characteristics in presenting Sysptoms(%)','Obtained value'],['Headache','43'],['Seizure','37'],['Confusion/memory','52'],['vision','23'],['Language','46'],['Motor','45'],['sensory','19']]

In [47]:
print('Symptoms baseline characteristics of all patients ')

In [48]:
print(tabulate(table3, headers='firstrow'))

In [49]:
table4=[['Characteristics ASA grade','Obtained Value'],['1','9'],['2','87'],['3','26'],['4','1']]

In [50]:
print('Baseline characteristics of ASA grade of all patients')

In [51]:
print(tabulate(table4, headers='firstrow'))

In [52]:
table5=[['Characteristics of Perioperative variables(%)','Obtained value'],['Steriod use','109'],['Preoperative dificit','116']]

In [53]:
print('Baseline characteristics of perioperative variables and postoperative deficit of all patients')

In [54]:
print(tabulate(table5, headers='firstrow'))

In [55]:
table6=[['Characteristics of Postoperative deficit','Obtained'],['None','46.1'],['New/Worsened','25.3'],['Improved','31.2']]

In [56]:
print(tabulate(table6, headers='firstrow'))

In [57]:
table7=[['Characteristics','Obtained value'],['Awake','16'],['Introperative stimulation','34'],['Biospy','47'],['CRET','58'],['PRET','17'],['EOR(%)[range]','99'],['RTVcvm3[IQR]','0.51']]

In [58]:
print('Baseline characteristics of surgical variables of all patients')

In [59]:
print(tabulate(table7, headers='firstrow'))

In [60]:
table8=[['characteristics','Obtained Value'],['IDH mutant','4'],['MGMT methylated','53']]

In [61]:
print('Baseline characteristics of tumor variables of all patients')

In [62]:
print(tabulate(table8, headers='firstrow'))

In [63]:
table9=[['Characteristics','Obtained value'],['Chemoradiotheraphy','79'],['Radiotherapy','29'],['None','18'],['Median OS(months)(95%CI)','10.4'],['Alive at last follow-up(%)','27']]

In [64]:
print('Baseline characteristics of Adjuvant theory (%) of all patients')

In [65]:
print(tabulate(table9, headers='firstrow'))

In [66]:
table10=[['-','Obtained IQR'],['WT','0.93'],['FLAIR','0.87'],['CER','0.93'],['NC','0.85']]

In [67]:
print('Median Dice Coefficients (MDC) for training')

In [68]:
print(tabulate(table10, headers='firstrow'))

In [69]:
table11=[['-','Obtained IQR (cm)'],['WT','79.0'],['FLAIR','42.6'],['CER','8.7'],['NC','8.9']]

In [70]:
print('Median Tumor Volume (MDV) for post-corrected (PC) segmentation for training')

In [71]:
print(tabulate(table11, headers='firstrow'))

In [72]:
table12=[['-','Obtained IQR(cm)'],['WT','84.1'],['Flair','62.9'],['CER','10.8'],['NC','7.9']]

In [73]:
print('Median Tumor Volume (MDV) for post-corrected (PC) segmentation for testing')

In [74]:
print(tabulate(table12, headers='firstrow'))

In [75]:
table13=[['Correlation','CER(R)','NC(R)','NET(r)'],['PTE','0.47','0.39','0.07'],['CER','-','0.43','-0.37'],['NET','-','-0.109','-']]

In [76]:
print('Correlation of PTE, CER, NET and NC')

In [77]:
print(tabulate(table13, headers='firstrow'))

In [78]:
table14=[['Correlation','PTE/NC'],['NET/TC(r)','0.31'],['NET/TC(r)','0.13']]

In [79]:
print('Correlation of PTE, CER, NET and NC')

In [80]:
print(tabulate(table14, headers='firstrow'))

In [81]:
print('Result of the survivial pedication in Brain tumor with the Brats dataset')

In [82]:
print(tabulate(table, headers='firstrow'))